In [ ]:
from pathlib import Path 
import pandas as pd 
import numpy as np
import datetime 
from IPython.display import HTML, display 
from multiprocessing import Pool
import os

In [ ]:
def load_new_arrests():
    base_path = Path.cwd()
    parsed_path = Path(base_path.parent,'MPD-Parser')
    parsed_files = sorted(parsed_path.rglob("newer*.csv"))
    df = pd.concat(pd.read_csv(f) for f in parsed_files)
    return df

df = load_new_arrests()
print(len(df))

In [ ]:
def clean_data(df):
    
    gender_mask = ~df['Gender'].isin(['F', 'M'])
    df.loc[gender_mask,'Gender'] = ''
    
    type_mask = ~df['Felony/Misdemeanor'].isin(['F', 'M'])
    df[type_mask] = ''

    titles = [ 'Offender First Name', 'Offender Last Name', 'Officer', 'Offense']
    for title in titles:
        df[title] = df[title].str.title()
        
    df['Arrest Date'][df['Arrest Date']=='Aug 5, 201 5 1:30:00 PM'] = 'Aug 5, 2015 1:30:00 PM'
    df['Arrest Date'][df['Arrest Date']=='Jun 5, 2015 10:10: 00 AM'] = 'Jun 5, 2015 10:10:00 AM'
    df['Arrest Date'][df['Arrest Date']=='Jun 10, 20158:39:00 PM'] = 'Jun 10, 2015 8:39:00 PM'
    
    df['Arrest Datetime'] = pd.to_datetime(df['Arrest Date'])
    df['Arrest Time'] = df['Arrest Datetime'].dt.time
    df['Arrest Date'] = df['Arrest Datetime'].dt.date

    for field in ['Offender First Name', 'Offender Last Name']:
        for char in [",", "`", "'", '"']:
            df[field] = df[field].str.replace(char, '').copy()

    # drop empty names
    df = df[df['Offender First Name'].notnull() & df['Offender Last Name'].notnull()].copy()
    print(len(df))
            
    # add full name
    df['Offender Full Name'] = df['Offender First Name'] + ' ' + df['Offender Last Name']
    # drop bad messages
    nonpersons = ['John Doe', 'Void', 'Jane Doe', 'Message', 'truncated', '<']
    for nonperson in nonpersons:
        mask = df['Offender Full Name'].str.contains(nonperson)
        df = df.loc[~mask]
        
    print(len(df))
    # drop duplicate records
    df.drop_duplicates(subset=['Arrest Date', 'Arrest Number', 'Offender Full Name', 'Offense'], inplace=True)        

    return df

print('cleaning data')
df = clean_data(df)
print('Done!', len(df), 'cleaned records.')

In [ ]:
today = datetime.date.today( )
yesterday = today - datetime.timedelta(days=-1)
date_range = pd.date_range(start='2015-09-01', end=yesterday)

def completeness_check(district_number):
    dates_with_data = 0
    for arrest_date in date_range:
        if len(df[(df['Arrest Date']==arrest_date.date()) & (df['PSA'].str[:1]==str(district_number))]) > 0:
            dates_with_data += 1
    print('District '+str(district_number)+'D has data for '+"{:.0%}".format(dates_with_data/len(date_range))+' of days since 2015-09-01')

district_numbers = list(range(1,8))
if __name__ == '__main__':
    pool = Pool(os.cpu_count()-1)              # Create a multiprocessing Pool
    pool.map(completeness_check, district_numbers)  # process iterable with pool


In [ ]:
df['DOB'] = pd.to_datetime(df['DOB'])
df['Age'] = pd.to_numeric(df['Age'], errors='coerce')
df['dob_range_start'] = df['Arrest Datetime'] - pd.to_timedelta(df['Age']+1, unit='Y') 
df['dob_range_start'] = df['dob_range_start'].dt.date
df['dob_range_end'] = df['Arrest Datetime'] - pd.to_timedelta(df['Age']-1, unit='Y')
df['dob_range_end'] = df['dob_range_end'].dt.date

df['ID']='' 
df = df.sort_values(by=['Offender Full Name', 'dob_range_start']).copy().reset_index(drop = True)

def progress(value, max=100):
    return HTML("""
        <progress
            value='{value}'
            max='{max}',
            style='width: 100%'
        >
            {value}
        </progress>
    """.format(value=value, max=max))


def disambiguate(df):
    out = display(progress(0, 100), display_id=True)
    total = len(df)
    ID_count=1
    for index in df.index:
        percent_done = round((index/total)*100)
        out.update(progress(percent_done, 100))
        if index==0:
            df.loc[index,'ID']=ID_count
        else:
            if df.loc[index,'Offender Full Name']==df.loc[index-1,'Offender Full Name']:
                if df.loc[index,'dob_range_start']<=df.loc[index-1,'dob_range_end']:
                    df.loc[index,'ID']=df.loc[index-1,'ID']
                else:
                    ID_count+=1
                    df.loc[index,'ID']=ID_count
            else:
                ID_count+=1
                df.loc[index,'ID']=ID_count
    return df


df1 = disambiguate(df)

In [ ]:
df1['name_id'] = df1['Offender Full Name'] + ' ' + df1['ID'].astype(str)
df2 = pd.DataFrame(columns=['min', 'max'])
df2['min'] = df1.groupby('name_id')['dob_range_start'].max()
df2['max'] = df1.groupby('name_id')['dob_range_end'].min()
print(len(df2.loc[df2['min']>df2['max']]), 'identities may contain deduping errors:')
df2.loc[df2['min']>df2['max']]


In [ ]:
df = df1.copy()

In [ ]:
dob_mask = df['DOB'].isnull()
df.loc[dob_mask,'DOB'] = df.loc[dob_mask,'dob_range_start']

In [ ]:
df = df.rename(index=str, columns={'ID':'consumer_id', 'Offender Full Name':'consumer_full_name',  'Offender First Name':'consumer_first_name', 'Offender Last Name':'consumer_last_name','Age':'consumer_age', 
                              'DOB':'consumer_dob', 'dob_range_start':'consumer_dob_start', 'dob_range_end':'consumer_dob_end', 
                              'Gender':'consumer_gender', 'Arrest Number':'event_id', 'Arrest Location':'event_location',
                              'Felony/Misdemeanor':'event_class', 'Offense':'event_type', 'Officer':'employee_id', 
                              'PSA':'employee_department', 'Arrest Datetime':'event_datetime', 'Arrest Date':'event_date', 
                              'Arrest Time':'event_time'}).copy()
df.columns

In [ ]:
fields = ['consumer_id', 'consumer_full_name', 'consumer_first_name', 'consumer_last_name', 'consumer_age', 'consumer_dob', 
          'consumer_dob_start', 'consumer_dob_end', 'consumer_gender', 
          'event_id', 'event_location', 'event_class', 'event_type', 'employee_id', 'employee_department', 
          'event_datetime', 'event_date', 'event_time']
reduced_fields = ['consumer_id', 'consumer_full_name', 'consumer_first_name', 
                  'consumer_last_name', 'consumer_dob', 'consumer_dob_start', 
                  'consumer_dob_end', 'consumer_gender', 'event_class', 'event_type', 'event_datetime']



In [ ]:
df.to_csv('cleaned_arrests.csv', index=False)

In [ ]:
len(df)